In [1]:
import tweepy
import pandas as pd
import time

In [26]:
import urllib.request

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/funding-dominance/data-collection/tweet_info.csv')
df['pfp url'] = ''
df['network size'] = ''

In [10]:
consumer_key = "cdV7bjdgWQSxMMVCoAVjUcFtu"
consumer_secret = "m0pUurMEa4vGqlkynbEBVFiOye9tjZSDsVlGadsPnF73iHvClA"
access_token = "1397642727517478918-SGxRcmFwS3zBDDCrPx8Vt4RfE93GOO"
access_token_secret = "FMYoaEGWpNrB6uJN8jhRu3L5GPYvGwWWk4FSvkynsyGpG"

In [7]:
users = []
for row in range(df.shape[0]):
  if df['tweets_period'][row] == 'ok' or df['tweets_period'][row] == 'hyperactive':
    users.append(df['usernames'][row])

In [42]:
def get_info(username):

  try:

    user_info = api.get_user(screen_name=username)

    url = user_info.profile_image_url
    url = url.replace('_normal','')
    url_https =user_info.profile_image_url_https
    network = user_info.followers_count

    return url, url_https, network

  except BaseException as e:
    print(username, '- failed on_status,',str(e))
    time.sleep(3)

In [30]:
data = pd.DataFrame(columns=['usernames', 'pfp_url', 'url_https', 'network_size'])
data.head()

,usernames,pfp_url,url_https,network_size


In [40]:
#adjust batch size:
batch_size = 100

n = len(users)
batches = n // batch_size
batch_rest = n % batches

print('batches =', batches, '|| batches_size =', batch_size, '|| batch_rest =', batch_rest)

batches = 16 || batches_size = 100 || batch_rest = 1


In [45]:
for batch in range(batches):
  start = batch * batch_size
  end = start + batch_size - 1

  print('Starting batch', batch)

  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_token_secret)
  api = tweepy.API(auth,wait_on_rate_limit=True)

  for i in range(start, end):
    username = users[i]
    if ' ' in username:
      pass
    else:
      url, url_https, network = get_info(username)
      row = {'usernames': username, 'pfp_url': url, 'url_https': url_https, 'network_size': network}
      data = data.append(row, ignore_index = True)

  print('Completed', batch)

username = users[-1]
url, url_https, network = get_info(username)
row = {'usernames': username, 'pfp_url': url, 'url_https': url_https, 'network_size': network}
data = data.append(row, ignore_index = True)

Starting batch 0
Completed 0
Starting batch 1
Completed 1
Starting batch 2
Completed 2
Starting batch 3
Completed 3
Starting batch 4
Completed 4
Starting batch 5
Completed 5
Starting batch 6
Completed 6
Starting batch 7
Completed 7
Starting batch 8
Completed 8
Starting batch 9
Completed 9
Starting batch 10
Completed 10
Starting batch 11
Completed 11
Starting batch 12
Completed 12
Starting batch 13
Completed 13
Starting batch 14
Completed 14
Starting batch 15
Completed 15


In [68]:
data.to_csv('/content/drive/MyDrive/funding-dominance/data-collection/img_network.csv')

In [ ]:
for i in range(len(users)):
  username = users[i]
  if ' ' in username:
    pass
  elif username in df['usernames'] and username in data['usernames']:
    index_df = df.index[df['usernames']==username]
    index_df = int(index_df[0])
    index_data = data.index[data['usernames']==username]
    index_data = int(index_data[0])

    df['pfp url'][index_df] = data['pfp_url'][index_data]
    df['network size'][index_df] = data['network_size'][index_data]

In [71]:
df.to_csv('/content/drive/MyDrive/funding-dominance/data-collection/twitter_info_meta.csv')

In [74]:
def save_pfp(url, username):
  try:
    filename = '/content/drive/MyDrive/funding-dominance/data-collection/pfps/{}.jpg'.format(username)
    urllib.request.urlretrieve(url,filename)
  except:
    print('Error for', username)

In [75]:
for i in range(data.shape[0]):
  url = data['pfp_url'][i]
  username = data['usernames'][i]
  save_pfp(url, username)

Error for mnicastro24
Error for persilientvj
Error for mnicastro24
Error for persilientvj
Error for mnicastro24
Error for persilientvj
Error for mnicastro24
Error for persilientvj
Error for davidpearah
Error for bechhoe1
Error for AdamTreiser
Error for Tenayshus_B
Error for RussFadel
Error for GadiRusc
Error for Gina_Gutierrez
Error for HenryWAdams
Error for Jan_Szilagyi
Error for mveerina
Error for RobertMcClung16
Error for gsmerdon
Error for akschultz_svt
Error for mrmiddleton
Error for jlotto
Error for rickcsong
Error for Michell67007246
Error for popstarr2000


In [76]:
errors = ['mnicastro24', 'persilientvj', 'davidpearah', 'bechhoe1', 'AdamTreiser', 'Tenayshus_B', 'RussFadel', 'GadiRusc',
          'Gina_Gutierrez', 'HenryWAdams', 'Jan_Szilagyi', 'mveerina', 'RobertMcClung16', 'gsmerdon', 'akschultz_svt', 'mrmiddleton',
          'jlotto', 'rickcsong', 'Michell67007246', 'popstarr2000']